In [ ]:
import os
import numpy as np
from load_data import *
from sklearn.model_selection import train_test_split
import torch
from torch.nn import *
from models import *
from training import *
from sklearn.model_selection import KFold
from torch.utils.data import *
import matplotlib.pyplot as plt
import time

In [ ]:
# Set torch seeds
torch.manual_seed(0)
torch.cuda.manual_seed(0)

# Load data

In [ ]:
params, solutions, basis_space, basis_time, sv_space, sv_time = load_data()

# Upper and lower bound of params
upper_bounds_params = np.array([8.0, 0.3, 0.5, 0.5, 0.5, 0.0])
lower_bounds_params = np.array([4.0, 0.1, -0.1, -0.5, -0.5, -0.3])

# Params mapping to [0, 1]
params = (params - lower_bounds_params) / (upper_bounds_params - lower_bounds_params)

# Split training and test set

In [ ]:
# Splitting train-test

split = 0.9

x_train_v, x_test_v, y_train_v, y_test_v = train_test_split(params, solutions['velocity'], test_size=1-split, random_state=0)
x_train_p, x_test_p, y_train_p, y_test_p = train_test_split(params, solutions['pressure'], test_size=1-split, random_state=0)

# Create Dataset class

In [ ]:
# Class for a dataset structure

class CreateDataset(Dataset):

    def __init__(self,x,y):
        self.x = torch.tensor(x, dtype = torch.float32)
        self.y = torch.tensor(y.reshape(x.shape[0], -1), dtype = torch.float32)  # unique tensor for space and time

    def __getitem__(self,index):
        sample = {
            'feature': self.x[index],
            'label': self.y[index]}
        return sample

    def __len__(self):
        return len(self.x)

# Cross validation

In [ ]:
def cross_validation(folds, x, y, PTCMap, K, num_epochs, lr, device):

    kf = KFold(folds)

    errors = []
    exec_times = []

    basis_space = y.shape[1]
    basis_time = y.shape[2]

    for fold_idx, (train_idx, val_idx) in enumerate(kf.split(x)):

        x_train, y_train, x_val, y_val = x[train_idx], y[train_idx], x[val_idx], y[val_idx]

        train = CreateDataset(x_train, y_train)
        val = CreateDataset(x_val, y_val)

        batch_size = 32

        torch.manual_seed(0)
        torch.cuda.manual_seed(0)

        train_gen = DataLoader(train, batch_size=batch_size, shuffle=True)
        val_gen = DataLoader(val, batch_size=batch_size, shuffle=True)

        model = PTCMap(K, basis_space, basis_time)

        start = time.time()

        run_training(train_gen, val_gen, num_epochs, model, lr, device)

        end = time.time()
        print('Execution time: {:.0f}s'.format(end - start))
        exec_times.append(end - start)

        error = compute_error(model, val_gen, device)
        errors.append(error)

    errors = np.array(errors)
    exec_times = np.array(exec_times)

    return np.mean(errors), np.std(errors), np.mean(exec_times), np.std(exec_times)

# Cross-validation for velocity

In [ ]:
# Parameters

# Set device for training
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Number of neurons per layer
K = 256

# Learning rate
lr = 0.01

# Number of epochs
num_epochs = 500

# K-fold
folds = 5

In [ ]:
error, std, t_mean, t_std = cross_validation(folds, x_train_v, y_train_v, PTCMapVelocity, K, num_epochs, lr, device)

In [ ]:
print("Error: mean = {:.3f}, std = {:.3f}".format(error, std))

In [ ]:
print("Time: mean = {:.3f}, std = {:.3f}".format(t_mean, t_std))

# Cross-validation for pressure

In [ ]:
# Parameters

# Set device for training
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Number of neurons per layer
K = 256

# Learning rate
lr = 0.01

# Number of epochs
num_epochs = 500

# K-fold
folds = 5

In [ ]:
error, std, t_mean, t_std = cross_validation(folds, x_train_p, y_train_p, PTCMapPressure, K, num_epochs, lr, device)

In [ ]:
print("Error: mean = {:.3f}, std = {:.3f}".format(error, std))

In [ ]:
print("Time: mean = {:.3f}, std = {:.3f}".format(t_mean, t_std))